In [1]:
!pip install ipython-autotime
%load_ext autotime

time: 2.17 ms (started: 2021-11-19 22:16:05 +00:00)


In [2]:
import pickle
import nltk
import pandas as pd
import numpy as np
from nltk.tokenize import TreebankWordTokenizer
from nltk.util import ngrams
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS as sklearn_stop_words
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

time: 671 ms (started: 2021-11-19 22:16:05 +00:00)


In [3]:
# Constants
# Pickle Input
JD_FILES_PICKLE_OUTPATH='/content/drive/MyDrive/AIDI1003/JDs/jds.pickle'
RESUME_FILES_PICKLE_OUTPATH='/content/drive/MyDrive/AIDI1003/Resumes/resumes.pickle'


time: 1.9 ms (started: 2021-11-19 22:16:06 +00:00)


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
time: 3.02 ms (started: 2021-11-19 22:16:06 +00:00)


In [5]:
jd_files_dict = resume_files_dict = {}
with open(JD_FILES_PICKLE_OUTPATH, 'rb') as fh:
  jd_files_dict = pickle.load(fh)
with open(RESUME_FILES_PICKLE_OUTPATH, 'rb') as fh:
  resume_files_dict = pickle.load(fh)

time: 1.01 s (started: 2021-11-19 22:16:06 +00:00)


In [6]:
nltk.download('stopwords')
stop_words = nltk.corpus.stopwords.words('english')
stop_words = sklearn_stop_words.intersection(stop_words)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
time: 44.9 ms (started: 2021-11-19 22:16:07 +00:00)


In [7]:
def tokenize(text):
  tokenized = text.lower().split()
  tokenized = [token for token in tokenized if (len(token)>1 and all(char.isalpha()or char.isdigit() for char in token))]
  return tokenized

time: 3.73 ms (started: 2021-11-19 22:16:07 +00:00)


In [8]:
print('Count of JDs:', len(jd_files_dict))
print('Count of Resumes:', len(resume_files_dict))

Count of JDs: 151210
Count of Resumes: 50023
time: 2.76 ms (started: 2021-11-19 22:16:07 +00:00)


In [9]:
combined_files_dict = {}
for filename in jd_files_dict:
  modified_fn = 'jd:' + filename
  if len(jd_files_dict[filename].strip()) == 0:
    continue
  combined_files_dict[modified_fn] = jd_files_dict[filename]
for filename in resume_files_dict:
  modified_fn = 'rs:' + filename
  if len(resume_files_dict[filename].strip()) == 0:
    continue
  combined_files_dict[modified_fn] = resume_files_dict[filename]

time: 265 ms (started: 2021-11-19 22:16:07 +00:00)


In [10]:
jd_filenames = [key for key in combined_files_dict.keys() if key.startswith('jd:')]

time: 46.5 ms (started: 2021-11-19 22:16:07 +00:00)


In [11]:
resume_filenames = [key for key in combined_files_dict.keys() if key.startswith('rs:')]

time: 40.8 ms (started: 2021-11-19 22:16:07 +00:00)


In [12]:
print('Count of Combines:', len(combined_files_dict))

Count of Combines: 201189
time: 2.76 ms (started: 2021-11-19 22:16:07 +00:00)


In [13]:
corpus_raw = pd.DataFrame.from_dict(combined_files_dict, orient='index', columns=['text'])


time: 97.8 ms (started: 2021-11-19 22:16:07 +00:00)


In [14]:
corpus_filenames = corpus_raw.index.values

time: 2.49 ms (started: 2021-11-19 22:16:08 +00:00)


In [15]:
corpus_filenames

array(['jd:_1158', 'jd:15x Bricklayers_87905', 'jd:_19361', ...,
       'rs:ZuellenTorres_37505', 'rs:ZuleikaFlores_24346',
       'rs:ZumbaInstructor_28782'], dtype=object)

time: 5.75 ms (started: 2021-11-19 22:16:08 +00:00)


In [16]:
vectorizer = TfidfVectorizer()
corpus_vectors = vectorizer.fit_transform(corpus_raw['text'])

time: 49.3 s (started: 2021-11-19 22:16:08 +00:00)


In [17]:
corpus_vectors.shape

(201189, 680560)

time: 3.6 ms (started: 2021-11-19 22:16:57 +00:00)


In [18]:
# Get sample value
# JD sample
jd_sample_index = np.where(corpus_filenames == 'jd:_19361')[0][0]
jd_test = corpus_vectors[jd_sample_index]

time: 8.23 ms (started: 2021-11-19 22:16:57 +00:00)


In [19]:
# Get sample value
# Resume sample
rs_sample_index = np.where(corpus_filenames == 'rs:ZuellenTorres_37505')[0][0]
rs_test = corpus_vectors[rs_sample_index]

time: 9.15 ms (started: 2021-11-19 22:16:57 +00:00)


In [20]:
jd_test_output = cosine_similarity(corpus_vectors, jd_test)

time: 572 ms (started: 2021-11-19 22:16:57 +00:00)


In [21]:
test = pd.DataFrame(jd_test_output, index = corpus_filenames, columns = ['similarity'])

time: 8.77 ms (started: 2021-11-19 22:16:58 +00:00)


In [22]:
# Get the largest similarity among all documents
test.nlargest(11, 'similarity')

,similarity
jd:_19361,1.000000
jd:Data Warehouse Ana_16208,0.321898
jd:Data Warehouse Ana_16199,0.316841
rs:PrudhviIndukuri_41849,0.313536
jd:Data Warehouse Ana_16211,0.311568
jd:Data Warehouse Ana_16248,0.309039
rs:SrOracleDatabaseAdministratorContractor_34291,0.307868
jd:Data Warehouse Ana_16246,0.307579
jd:Oracle Database Ad_22937,0.307514
jd:Database Administr_59540,0.305045


time: 24.7 ms (started: 2021-11-19 22:16:58 +00:00)


In [23]:
# Get the largest similarity of only JDs
test_jd_only = test.loc[ jd_filenames, : ]

time: 166 ms (started: 2021-11-19 22:16:58 +00:00)


In [24]:
test_jd_only

,similarity
jd:_1158,0.128046
jd:15x Bricklayers_87905,0.060180
jd:_19361,1.000000
jd:1C Developer_15174,0.084923
jd:1C Developer_5043,0.077787
...,...
jd:Zscaler Security E_23331,0.120874
jd:ZSM DSL_70369,0.100540
jd:ZURICH FAST SDE_2333,0.154837
jd:Zurich SDM CREST_2366,0.074377


time: 16.9 ms (started: 2021-11-19 22:16:58 +00:00)


In [25]:
test_jd_only.nlargest(11, 'similarity')

,similarity
jd:_19361,1.000000
jd:Data Warehouse Ana_16208,0.321898
jd:Data Warehouse Ana_16199,0.316841
jd:Data Warehouse Ana_16211,0.311568
jd:Data Warehouse Ana_16248,0.309039
jd:Data Warehouse Ana_16246,0.307579
jd:Oracle Database Ad_22937,0.307514
jd:Database Administr_59540,0.305045
jd:Senior Oracle PLSQ_31395,0.304796
jd:Senior Oracle Uni_68287,0.304301


time: 22 ms (started: 2021-11-19 22:16:58 +00:00)


In [26]:
# Get the largest similarity of only Resumes
test_rs_only = test.loc[ resume_filenames, : ]

time: 79.2 ms (started: 2021-11-19 22:16:58 +00:00)


In [27]:
test_rs_only.nlargest(11, 'similarity')

,similarity
rs:PrudhviIndukuri_41849,0.313536
rs:SrOracleDatabaseAdministratorContractor_34291,0.307868
rs:GilbertBaltimore_32445,0.304036
rs:TusharReddy_23622,0.298362
rs:VenkatRamaiah_14037,0.298120
rs:KVarma_13433,0.290773
rs:LindaStewart_1461,0.288696
rs:PLSQLETLDeveloper_25030,0.287903
rs:TechnicalAnalystDeveloperModuleLead_35805,0.283132
rs:LeadOracleDeveloper_11171,0.282850


time: 14.7 ms (started: 2021-11-19 22:16:58 +00:00)
